# 数据集采集与网络爬虫：获取股票交易行情数据


## 实习任务和要求

1、访问

http://quote.eastmoney.com/center/gridlist.html#hs_a_board

获取“沪深A股”的交易行情数据。

注意：在上图红框标注的“沪深A股”标签页的数据共有194页，要解决翻页的问题，

要下载所有的数据，并保存数据到数据库（使用任何一种数据库）中。

2、访问

http://www.swsindex.com/idx0530.aspx#

下载沪深A股“全部行业分类”数据。是一个Excel文件。

把这个Excel文件的数据导入到数据库中。



3、访问类似如下的网页

http://data.eastmoney.com/zjlx/002157.html


注意，上述链接中标为红色的是股票代码，它可以同第二步中的Excel文件中提取，或者从第一步中下载的到的数据中提取。

要求：下载所有股票的历史资金流向数据，并保存在数据库中。

4、根据以上下载的数据，做如下的数据分析。

4.1、根据第二步中的行业分类，统计得出每个行业的资金流向数据。并用合适的可视化方式展现出来。

4.2、根据上述采集的数据，试分析一个股票的资金流入性质的不同统计数据与股票涨跌之间有没有某种关系？

4.3、自己设计和证券相关的一个问题，并下载数据进行分析。（必做，占30%的成绩）。


## 一、获取页面

In [68]:
import requests
from  bs4 import BeautifulSoup
import os
import lxml
import json
import time

In [106]:

#获取url下的页面内容，返回soup对象
def get_page(url):
    responce = requests.get(url)
    soup = BeautifulSoup(responce.text,'lxml')
    return soup




## 二、解析数据

In [124]:
def analysis_soup(soup):
    """
    传入一个suop对象，对其进行解析得到股票实时信息数据的字典列表
    """
    #获取P标签下的数据
    a=soup.find_all('p')
    #类型转换
    b = str(a)
    #对字符串进行分割，去掉不需要的数据"<p>jQuery112408123510576841297_1577064081282();</p>"
    c=b.split(">")
    #得到json格式的str
    d=c[1].split("<")
    #用json对数据进行解析
    e = json.loads(d[0])
    #得到data下的字典数据
    f=e["data"]

    #得到列表g,存储股票信息的字典
    g = f["diff"]
    return g

## 三、获取所有数据

In [126]:
def get_shares_data(): 
    """获取所有股票信息"""
    shares_data_list= []
    for i in range(1,195):
        
        t =time.time()
        nowTime = lambda:int(round(t * 1000))           #毫秒级时间戳，基于lambda
        #i是页面
        url ="""http://39.push2.eastmoney.com/api/qt/clist/get?cb=&pn="""+str(i)+"""
        &pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2
        &fid=f3&fs=m:0+t:6,m:0+t:13,m:0+t:80,m:1+t:2,m:1+t:23&
        fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152
        &_=1"""+str(nowTime)

        #获取网页信息
        soup =  get_page(url)
        #获取每页的数据
        shares_page_data = analysis_soup(soup)
        shares_data_list[len(shares_data_list):len(shares_data_list)]= shares_page_data
    return shares_data_list

In [114]:
shares_data_list = get_shares_data()

In [115]:
len(shares_data_list)

3875

## 四、数据库操作